In [1]:
using Pkg
Pkg.activate("/Users/ryansenne/Documents/GitHub/SSM")

include("/Users/ryansenne/Documents/GitHub/SSM//src/SSM.jl")
using .SSM
using LinearAlgebra
using Random
using Distributions
using Plots
using CSV
using DataFrames
using BenchmarkTools

  Activating project at `\Users\ryansenne\Documents\GitHub\SSM`


In [ ]:
# Generate data from a known dynamical system--a pendulum
g = 9.81 # gravity
l = 1.0 # length of pendulum
dt = 0.001 # time step
T = 0.004 # total time
# Discrete-time dynamics
A = [1.0 dt; -g/l*dt 1.0]
# Initial state
x0 = [0.0; 1.0]
# Time vector
t = 0:dt:T
# Generate data
x = zeros(2, length(t))
x[:,1] = x0
for i = 2:length(t)
    x[:,i] = A*x[:,i-1]
end

# Now lets add noise to the system and see if we can recover the dynamics
# Add driving Gaussian noise to simulate observations
x_noisy = zeros(2, length(t))
x_noisy[:, 1] = x0
observation_noise_std = 0.5

noise = rand(Normal(0, observation_noise_std), (2, length(t)))

for i in 2:length(t)
    x_noisy[:, i] = A * x[:, i-1] + noise[:, i]
end

# Define the LDS model parameters
H = I(2)  # Observation matrix (assuming direct observation)
Q = 1e-8 * I(2)  # Process noise covariance
R = 0.25 * I(2)  # Observation noise covariance
P0 = 1e-2 * I(2)  # Initial state covariance
x0 = [0.0; 1.0]  # Initial state mean

# Create the Kalman filter parameter vector
kf = LDS(A, Matrix{Float64}(H), nothing, Matrix{Float64}(Q), Matrix{Float64}(R), x0, Matrix{Float64}(P0), nothing, 2, 2, ector([true, true, false, true, true, true, true, false]))

# Run the Kalman filter
x_filt, p_filt, x_pred, p_pred, v, F, K, ll = KalmanFilter(kf, Matrix{Float64}(x_noisy'))
# Run the smoother
x_smooth, P_smooth = KalmanSmoother(kf, Matrix{Float64}(x_noisy'))

In [9]:
# calculate the predictions of X
using Profile
x_prediction = zeros(length(t), 2)
x_prediction[1, :] = kf.x0
for t in 2:length(t)
    x_prediction[t, :] = kf.A * x_prediction[t-1, :]
end

@profile x_new = SSM.newton_raphson_tridg!(kf, x_prediction, x_noisy', 10, 1e-3)

UndefVarError: UndefVarError: `kf` not defined

In [13]:
using ForwardDiff
# H = ForwardDiff.hessian(x -> SSM.loglikelihood(x', kf, x_noisy[1:2, 1:100]'), x_prediction')
G = ForwardDiff.gradient(x -> SSM.loglikelihood(x', kf, x_noisy'), x_new')
G = reshape(G, 10, 1)

UndefVarError: UndefVarError: `x_new` not defined

In [7]:
# need to use previously aquired pendulum data for comparison
pendulum_data = CSV.read("/Users/ryansenne/Documents/GitHub/SSM/notebooks/pendulum_data_1.csv", DataFrame)

# convert x_noisy columns to matrix for use 
x_noisy =  Matrix(pendulum_data[1:10, 3:4])

# now create a Kalman Filter where we want to learn the parameters
A = [0.1 0.2; 0.3 0.4]
H = [1 0.0; 0.0 1]
Q = 1 * I(2)
R = 1 * I(2)
p0 = 1 * I(2)
x0 = [0.0, 1.0]

# kf = LDS(A, Matrix{Float64}(H), nothing, Matrix{Float64}(Q), Matrix{Float64}(R), x0, Matrix{Float64}(P0), nothing, 2, 2, "Gaussian", Vector([true, true, false, true, true, true, true, false]))
kf_unknown = LDS(A, H, nothing, Q, R, x0, p0, nothing, 2, 2, [true, true, true, true, true, true])
# SSM.pca_init!(kf_unknown, x_noisy)

SSM.KalmanFilterEM!(kf_unknown, Matrix{Float64}(x_noisy), 100)

Log-likelihood at iteration 1: -18.52416478401385
Log-likelihood at iteration 2: -7.990402782400094
Log-likelihood at iteration 3: -5.2683120641671835
Log-likelihood at iteration 4: -2.950020329353198
Log-likelihood at iteration 5: -1.4362789886824838
Log-likelihood at iteration 6: -0.5831952403829748
Log-likelihood at iteration 7: -0.10149535390573794
Log-likelihood at iteration 8: 0.23883769486552187
Log-likelihood at iteration 9: 0.5449513014075665
Log-likelihood at iteration 10: 0.835997801659804
Log-likelihood at iteration 11: 1.0965240155787668
Log-likelihood at iteration 12: 1.3146652596502049
Log-likelihood at iteration 13: 1.4925552977574406
Log-likelihood at iteration 14: 1.6392075378301996
Log-likelihood at iteration 15: 1.7633783155448162
Log-likelihood at iteration 16: 1.871302083761674
Log-likelihood at iteration 17: 1.966971600285421
Log-likelihood at iteration 18: 2.052923248624639
Log-likelihood at iteration 19: 2.130846014733386
Log-likelihood at iteration 20: 2.20194

Log-likelihood at iteration 23: 2.382710487506915
Log-likelihood at iteration 24: 2.434226583236436
Log-likelihood at iteration 25: 2.4822070580179934
Log-likelihood at iteration 26: 2.52705928734588
Log-likelihood at iteration 27: 2.569144510117078
Log-likelihood at iteration 28: 2.608785477564656
Log-likelihood at iteration 29: 2.6462726116954904
Log-likelihood at iteration 30: 2.6818690967535206
Log-likelihood at iteration 31: 2.715815165069909
Log-likelihood at iteration 32: 2.7483317509858702
Log-likelihood at iteration 33: 2.779623634064952
Log-likelihood at iteration 34: 2.809882157565098
Log-likelihood at iteration 35: 2.839287581765398
Log-likelihood at iteration 36: 2.868011110229478
Log-likelihood at iteration 37: 2.896216608482021
Log-likelihood at iteration 38: 2.924062018034876
Log-likelihood at iteration 39: 2.9517004540966907
Log-likelihood at iteration 40: 2.9792809631736072
Log-likelihood at iteration 41: 3.006948908272053
Log-likelihood at iteration 42: 3.03484594631

Log-likelihood at iteration 60: 3.6824578599007887
Log-likelihood at iteration 61: 3.7276719781290772
Log-likelihood at iteration 62: 3.773523261311382
Log-likelihood at iteration 63: 3.8199345931998057
Log-likelihood at iteration 64: 3.8668329456771375
Log-likelihood at iteration 65: 3.914150405176045
Log-likelihood at iteration 66: 3.96182482494466
Log-likelihood at iteration 67: 4.009800153086844
Log-likelihood at iteration 68: 4.058026492945886
Log-likelihood at iteration 69: 4.1064599534295745
Log-likelihood at iteration 70: 4.155062343908339
Log-likelihood at iteration 71: 4.203800762749817
Log-likelihood at iteration 72: 4.25264712153426
Log-likelihood at iteration 73: 4.301577639415365


Log-likelihood at iteration 74: 4.350572334597928
Log-likelihood at iteration 75: 4.399614532934363
Log-likelihood at iteration 76: 4.448690407496795
Log-likelihood at iteration 77: 4.497788557786712
Log-likelihood at iteration 78: 4.546899633051408
Log-likelihood at iteration 79: 4.596016000950456
Log-likelihood at iteration 80: 4.645131460463995
Log-likelihood at iteration 81: 4.694240996336821
Log-likelihood at iteration 82: 4.743340571380962
Log-likelihood at iteration 83: 4.792426952468861
Log-likelihood at iteration 84: 4.841497565923984
Log-likelihood at iteration 85: 4.890550378142736
Log-likelihood at iteration 86: 4.939583797573093
Log-likelihood at iteration 87: 4.988596594556411
Log-likelihood at iteration 88: 5.03758783596391
Log-likelihood at iteration 89: 5.0865568319853605
Log-likelihood at iteration 90: 5.135503092831577
Log-likelihood at iteration 91: 5.184426293480691
Log-likelihood at iteration 92: 5.233326244927835
Log-likelihood at iteration 93: 5.282202870674976

5.574978455679801
Log-likelihood at iteration 100: 5.6236957909577825


(LDS([-0.30347035021863217 0.11714450210987123; 0.10259424393111494 0.9730242962817144], [0.2903580638604299 -0.191059849057932; 0.1995781043798704 0.5041376165573054], nothing, [1.7428426335202243 -0.09614114619032993; -0.09614114619032993 0.006313597398528437], [0.01914088403511903 -0.03034701363592889; -0.03034701363592889 0.049408734739383044], [0.9817541531574818, 1.7644164393906965], [0.0003390913022373043 -0.0001590485758111715; -0.0001590485758111715 0.0004655070686760965], nothing, 2, 2, Bool[1, 1, 1, 1, 1, 1]), 5.6236957909577825)

In [15]:
x_filt, p_filt, x_pred, p_pred, v, S, K, ll =SSM.KalmanFilter(kf_unknown, Matrix{Float64}(x_noisy))

([0.0 1.0; 0.22566239984008948 0.45610640028696026; … ; -0.08326931777853783 -0.04415219514174039; -0.015233336622534323 0.0071783586296058285], [0.5 0.0; -0.02159198891631108 -0.006507611970760366; … ; -0.06016781257525112 -0.0807295298198789; -0.060184103980500656 -0.08076049549473423;;; 0.0 0.5; -0.006507611970760364 0.1163386778832134; … ; -0.0807295298198789 -0.02941233463191853; -0.08076049549473423 -0.029471192232264215], [0.0 1.0; 0.2 0.4; … ; -0.01461213073443553 -0.034279829983049084; -0.01715737080620186 -0.042641673390257505], [1.0 0.0; -0.02108970643722805 -0.007209060966286943; … ; -0.05108643781697929 -0.07441658035128658; -0.05109705914105246 -0.07444003509608585;;; 0.0 1.0; -0.007209060966286943 0.1317083691926869; … ; -0.07441658035128658 -0.0227360002420455; -0.07444003509608585 -0.0227877946369636], [0.0 0.0; -1.3117510767744314 0.4088926756141953; … ; -0.25747359019323585 1.0423547610109465; -0.8353662184963082 0.5987052565231548], [2.0 0.0; 0.9789102935627719 -0.0

In [28]:
for i in 1:10
    z = ((kf_unknown.H * p_pred[i, :, :] * kf_unknown.H') + kf_unknown.R)
    println(det(z))
end

4.0
1.1077890013538432
0.9760529538224801
0.9398591439616736
0.9280005251778541
0.9239155790544029
0.9224841152765938
0.9219794969436195
0.9218012357723678
0.9217382166871365


In [21]:
p_filt[4, :, :]

2×2 Matrix{Float64}:
 -0.0555903  -0.072028
 -0.072028   -0.0128714

In [3]:
kf_unknown.R

2×2 Matrix{Float64}:
  0.300558  -0.170841
 -0.170841   0.387198